In [1]:
import re

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import matplotlib
from matplotlib import cm
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [2]:
# dev
pd.set_option('display.max_rows', 500)

In [3]:
# read initial data
df = pd.read_csv('/Users/kutch/nfl/2018_data.csv'
                 ,index_col=False
                 ,low_memory=False)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# global dash attributes
app = dash.Dash(__name__)
app.title = "NFL Plotting with Dash"

In [5]:
# make a green->yellow->red colormap for plotly/dash
grrd_cmap = matplotlib.cm.get_cmap('RdYlGn').reversed()
grrd_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)
for i in range(0, 255):
    k = matplotlib.colors.colorConverter.to_rgb(grrd_cmap(norm(i)))
    grrd_rgb.append(k)
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale
grrd = matplotlib_to_plotly(grrd_cmap, 255)

In [6]:
team_abbrevs = {'ARI': 'Arizona Cardinals'
               ,'ATL': 'Atlanta Falcons'
               ,'BAL': 'Baltimore Ravens'
               ,'BUF': 'Buffalo Bills'
               ,'CAR': 'Carolina Panthers'
               ,'CHI': 'Chicago Bears'
               ,'CIN': 'Cincinnati Bengals'
               ,'CLE': 'Cleveland Browns'
               ,'DAL': 'Dallas Cowboys'
               ,'DEN': 'Denver Broncos'
               ,'DET': 'Detroit Lions'
               ,'GB': 'Green Bay Packers'
               ,'HOU': 'Houston Texans'
               ,'IND': 'Indianapolis Colts'
               ,'JAX': 'Jacksonville Jaguars'
               ,'KC': 'Kansas City Chiefs'
               ,'LA': 'Los Angeles Rams'
               ,'LAC': 'Los Angeles Chargers'
               ,'MIA': 'Miami Dolphins'
               ,'MIN': 'Minnesota Vikings'
               ,'NE': 'New England Patriots'
               ,'NO': 'New Orleans Saints'
               ,'NYG': 'New York Giants'
               ,'NYJ': 'New York Jets'
               ,'OAK': 'Oakland Raiders'
               ,'PHI': 'Philadelphia Eagles'
               ,'PIT': 'Pittsburgh Steelers'
               ,'SEA': 'Seattle Seahawks'
               ,'SF': 'San Francisco 49ers'
               ,'TB': 'Tampa Bay Buccaneers'
               ,'TEN': 'Tennessee Titans'
               ,'WAS': 'Washington Redskins'}

In [7]:
# styling
forestgreen = '#012800'
green = '#024e00'
lightgray = '#cfcfcf'
white = '#fff'
colors = {'background': forestgreen
          ,'text': lightgray
         }

In [8]:
# dataframes for different graph options
## penalty dataframe
df_pen = df[[col for col in df.columns if re.search("(penalty)|(game\_date)", col) or col in ('posteam', 'defteam')]]
df_pen['penalty_side'] = np.where(df['posteam']==df['penalty_team'], 'Offensive', 'Defensive')
df_pen = df_pen[df_pen['penalty']==1.0].drop(['penalty_player_id'
                                             ,'penalty'
                                             ,'posteam'
                                             ,'defteam']
                                            ,axis=1)
df_pen['penalty_type'].replace(to_replace=np.nan, value='Not Recorded', inplace=True)
df_pen = df_pen.sort_values(['game_date'
                             ,'penalty_team'
                             ,'penalty_player_name'
                             ,'penalty_yards'])
df_pen = df_pen.reset_index(drop=True)

In [33]:
### penalty breakouts
pen_by_team = df_pen.groupby(['penalty_team'], as_index=False).sum().sort_values(['penalty_yards'])
pen_by_team_offdef = df_pen.groupby(['penalty_team', 'penalty_side'], as_index=False).sum().sort_values(['penalty_yards'])
pen_by_type = df_pen.groupby(['penalty_team', 'penalty_side', 'penalty_type'], as_index=False).sum().sort_values(['penalty_yards'])
penalty_types = pen_by_type.drop_duplicates(subset=['penalty_side', 'penalty_type'])[['penalty_side','penalty_type']].sort_values(['penalty_side','penalty_type'])
penalty_types = penalty_types.reset_index(drop=True)
pen_by_team_game = df_pen.groupby(['penalty_team','game_date'], as_index=False).sum()
pen_by_player = df_pen.groupby(['penalty_player_name'], as_index=False).sum()

In [ ]:
# penalty graph
app.layout = html.Div([
    html.Div(id='page-content', children=[
        html.H1(children="2018 NFL Play Breakdown")
        ,dcc.Graph(id='graph_pen_by_team'
                   ,figure={'data': []} )
        ,dcc.Dropdown(id='team_dropdown'
                      ,options=[{'label': team_abbrevs[team], 'value': team} for team in pen_by_team['penalty_team'].unique()]
                      ,value=[team for team in pen_by_team['penalty_team'].unique()]
                      ,multi=True
                      ,style={'background-color': '#012800'
                              ,'color': '#cfcfcf'}
                      ,placeholder="Filter by team(s)" )
        ,dcc.RadioItems(
                id='penalty-side-radio'
                ,options=[{'label': i, 'value': i} for i in
                         ['All', 'Offensive', 'Defensive']]
                ,value='All'
                ,labelStyle={'display': 'inline-block'} )
        ,dcc.Dropdown(id='penalty-type-dropdown'
                      ,options=[{'label': pen_type, 'value': pen_type} for pen_type in pen_by_type['penalty_type'].unique()]
                      ,value=[pen_type for pen_type in pen_by_type['penalty_type'].unique()]
                      ,multi=True
                      ,style={'background-color': '#012800'
                              ,'color': '#cfcfcf'}
                      ,placeholder="Type of penalty" )
    ])
])

@app.callback(
    dash.dependencies.Output('graph_pen_by_team', 'figure')
    ,[dash.dependencies.Input('team_dropdown', 'value')]
    ,[dash.dependencies.State('penalty-side-radio', 'value')] )
    #,[dash.dependencies.Input('penalty-type-dropdown', 'value')] )
def team_penalty_figure(team_values, side_value):#, type_values):
    traces = []
    title = "<b>Penalty Dashboard</b>"
    #print(type_values)
    if side_value == 'All':
        filtered_df = pen_by_team[pen_by_team['penalty_team'].isin(team_values)]
    elif side_value in (['Offensive','Defensive']):
        filtered_df = pen_by_team_offdef[(pen_by_team_offdef['penalty_team'].isin(team_values))
                                       & (pen_by_team_offdef['penalty_side']==side_value)]
        title += f'- {side_value}'
    else:
        raise ValueError
    traces.append(go.Bar(x=filtered_df['penalty_team']
                        ,y=filtered_df['penalty_yards']
                        ,name='Selected Teams'
                        ,marker={'color': filtered_df['penalty_yards']
                                ,'colorscale': grrd}))
    return {'data': traces
        ,'layout': {'plot_bgcolor': colors['text']
                    ,'paper_bgcolor': colors['text']
                    ,'title': title
                    ,'titlefont':dict(size=24, color='#565656', family='Arial, sans-serif')
                   } }

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)